# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""15096273807482…",1.2927e19,"""10415247725929…","""../../../../da…","""12648062268803…","""99287446608352…",false
"""61257060577359…",1.4856e19,"""13111856708695…","""../../../../da…","""12648062268803…","""12356567898157…",true
"""11902218263299…",5.6458e17,"""16517698360900…","""../../../../da…","""63765380661399…","""99287446608352…",false
"""61257060577359…",4.0723e18,"""12519793148166…","""../../../../da…","""16611727256486…","""99287446608352…",false
"""15096273807482…",1.4677e19,"""72518800370992…","""../../../../da…","""16611727256486…","""15652857007510…",true
"""11902218263299…",1.6330e19,"""25036157208598…","""../../../../da…","""16501365737861…",null,false
"""11902218263299…",8.7675e18,"""14941435984849…","""../../../../da…","""16611727256486…","""15652857007510…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""23029588801737…",1.2927e19,"""12211140240965…","""../../../../da…","""FirstMortgage3…","""68556019509914…","""Rejected"""
"""14177360389680…",1.4856e19,"""10171804429325…","""../../../../da…","""FirstMortgage3…","""26073375048139…","""Accepted"""
"""10442335066897…",5.6458e17,"""15700251806466…","""../../../../da…","""MoneyMarketSav…","""68556019509914…","""Rejected"""
"""14177360389680…",4.0723e18,"""10909096629321…","""../../../../da…","""BasicChecking""","""68556019509914…","""Rejected"""
"""23029588801737…",1.4677e19,"""51873974850132…","""../../../../da…","""BasicChecking""","""11290065006489…","""Accepted"""
"""10442335066897…",1.6330e19,"""11148123836310…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""10442335066897…",8.7675e18,"""18367806767731…","""../../../../da…","""BasicChecking""","""11290065006489…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""14566395798993…",1.2927e19,"""13663805578124…","""../../../../da…","""FirstMortgage3…","""11068702459983…","""Rejected"""
"""25716415769508…",1.4856e19,"""12088001521893…","""../../../../da…","""FirstMortgage3…","""14460625516836…","""Accepted"""
"""25229025346425…",5.6458e17,"""23497766869994…","""../../../../da…","""MoneyMarketSav…","""11068702459983…","""Rejected"""
"""25716415769508…",4.0723e18,"""10919314726274…","""../../../../da…","""BasicChecking""","""11068702459983…","""Rejected"""
"""14566395798993…",1.4677e19,"""13950346289600…","""../../../../da…","""BasicChecking""","""11636884064162…","""Accepted"""
"""25229025346425…",1.6330e19,"""10785687785651…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""25229025346425…",8.7675e18,"""14595475344767…","""../../../../da…","""BasicChecking""","""11636884064162…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_CLV=PREDICTOR_1
Customer_City=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""42992054441753…",1.2927e19,"""16251739567948…","""FirstMortgage3…","""93030672948724…","""Rejected"""
"""16536164128611…",1.4856e19,"""20524772880319…","""FirstMortgage3…","""42115687952374…","""Accepted"""
"""21363071428431…",5.6458e17,"""10103772658414…","""MoneyMarketSav…","""93030672948724…","""Rejected"""
"""16536164128611…",4.0723e18,"""86381634037659…","""BasicChecking""","""93030672948724…","""Rejected"""
"""42992054441753…",1.4677e19,"""68733081511576…","""BasicChecking""","""98783608240385…","""Accepted"""
"""21363071428431…",1.6330e19,"""10959888224800…","""UPlusFinPerson…",null,"""Rejected"""
"""21363071428431…",8.7675e18,"""29026374843739…","""BasicChecking""","""98783608240385…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'